# Wolfram Elementary Cellular Automata

This notebook implements [Wolfram's Elementary Cellular Automata](https://mathworld.wolfram.com/ElementaryCellularAutomaton.html) - one-dimensional cellular automata where each cell's next state depends on its current state and its two immediate neighbors.

With 3 cells determining the next state (left neighbor, self, right neighbor), there are 2³ = 8 possible input configurations. Each rule (0-255) specifies an output bit for each configuration, giving us 2⁸ = 256 possible rules.

## Discrete Calculus Framework

We decompose the CA evolution using a discrete calculus:

- **G** - Configuration space
- **R** - Ruleset (maps neighborhoods to outputs)
- **N** - Neighborhood (left, center, right)
- **S** - State vector

**Core Operations:**
- **D(S)** - Derivative: `D[i] = R(N[i]) XOR S[i]` — cells that will change
- **I(S, D)** - Integral: `S XOR D` — flip cells where derivative is 1
- **E(S)** - Evolve: `I(S, D(S))` — the step function decomposed
- **C(S)** - Commutator: `E(D(S)) XOR D(E(S))` — reveals nonlinearity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional
from IPython.display import display

## Core Implementation

The `WolframCA` class encapsulates the cellular automaton logic.

In [ ]:
class WolframCA:
    """Elementary Cellular Automaton with discrete calculus operations."""
    
    def __init__(self, rule: int, width: int = 101):
        """
        Initialize a Wolfram CA.
        
        Args:
            rule: Rule number (0-255)
            width: Width of the CA grid
        """
        if not 0 <= rule <= 255:
            raise ValueError("Rule must be between 0 and 255")
        
        self.rule = rule
        self.width = width
        self.R = self._build_rule_table(rule)  # R: the ruleset
        self.history: list[np.ndarray] = []
    
    def _build_rule_table(self, rule: int) -> dict[tuple[int, int, int], int]:
        """Build lookup table R for the rule."""
        table = {}
        for i in range(8):
            output = (rule >> i) & 1
            left = (i >> 2) & 1
            center = (i >> 1) & 1
            right = i & 1
            table[(left, center, right)] = output
        return table
    
    def N(self, state: np.ndarray, i: int) -> tuple[int, int, int]:
        """
        Get the neighborhood N at position i.
        
        Args:
            state: Current state
            i: Cell position
            
        Returns:
            Tuple (left, center, right)
        """
        return (
            state[(i - 1) % self.width],
            state[i],
            state[(i + 1) % self.width]
        )
    
    def initialize(self, initial_state: Optional[np.ndarray] = None) -> np.ndarray:
        """
        Initialize the CA with a starting state.
        
        Args:
            initial_state: Custom initial state, or None for single center cell
        
        Returns:
            The initial state array
        """
        if initial_state is not None:
            state = initial_state.copy()
        else:
            state = np.zeros(self.width, dtype=np.uint8)
            state[self.width // 2] = 1
        
        self.history = [state]
        return state
    
    def differentiate(self, state: np.ndarray) -> np.ndarray:
        """
        Compute the derivative D(S).
        
        D(S)[i] = R(N[i]) XOR S[i]
        
        The derivative is a bitmask where cells are 1 wherever the state
        will change under evolution. Defined directly in terms of R.
        
        Args:
            state: Current state (1D binary array)
            
        Returns:
            Derivative bitmask (same shape as state)
        """
        derivative = np.zeros_like(state)
        for i in range(self.width):
            neighborhood = self.N(state, i)
            rule_output = self.R[neighborhood]
            derivative[i] = rule_output ^ state[i]
        return derivative
    
    def integrate(self, state: np.ndarray, derivative: np.ndarray) -> np.ndarray:
        """
        Compute the integral I(S, D).
        
        I(S, D) = S XOR D
        
        Flips cells in S wherever D is 1.
        
        Args:
            state: Current state (1D binary array)
            derivative: Derivative bitmask (1D binary array)
            
        Returns:
            New state with cells flipped where derivative is 1
        """
        return np.bitwise_xor(state, derivative).astype(np.uint8)
    
    def step(self) -> np.ndarray:
        """
        Advance the CA by one generation.
        
        E(S) = I(S, D(S)) = S XOR D(S)
        
        The step function decomposed through differentiate and integrate.
        """
        if not self.history:
            self.initialize()
        
        current = self.history[-1]
        d = self.differentiate(current)
        next_state = self.integrate(current, d)
        
        self.history.append(next_state)
        return next_state
    
    def evolve(self, state: np.ndarray) -> np.ndarray:
        """
        Compute E(S) = I(S, D(S)) without modifying history.
        
        Args:
            state: Current state (1D binary array)
            
        Returns:
            Next state
        """
        d = self.differentiate(state)
        return self.integrate(state, d)
    
    def commutator(self, state: np.ndarray) -> np.ndarray:
        """
        Compute the commutator C(S) = E(D(S)) XOR D(E(S)).
        
        The commutator reveals nonlinearity - cells are 1 where the
        order of operations between E and D matters.
        
        Args:
            state: Current state (1D binary array)
            
        Returns:
            Commutator bitmask showing positions of nonlinearity
        """
        # E(D(S)) - evolve the derivative
        d_s = self.differentiate(state)
        e_d_s = self.evolve(d_s)
        
        # D(E(S)) - derivative of the evolved state
        e_s = self.evolve(state)
        d_e_s = self.differentiate(e_s)
        
        # Commutator: where these differ
        return np.bitwise_xor(e_d_s, d_e_s).astype(np.uint8)
    
    def run(self, generations: int) -> np.ndarray:
        """
        Run the CA for a specified number of generations.
        
        Args:
            generations: Number of generations to simulate
        
        Returns:
            2D array of the complete history
        """
        if not self.history:
            self.initialize()
        
        for _ in range(generations):
            self.step()
        
        return np.array(self.history)
    
    def get_history(self) -> np.ndarray:
        """Return the full history as a 2D numpy array."""
        return np.array(self.history)
    
    def reset(self):
        """Clear the history."""
        self.history = []
    
    def __repr__(self) -> str:
        return f"WolframCA(rule={self.rule}, width={self.width})"

## Visualization

In [ ]:
def plot_ca(ca: WolframCA, figsize: tuple[int, int] = (12, 8), cmap: str = 'binary'):
    """
    Plot the CA evolution.
    
    Args:
        ca: The WolframCA instance to plot
        figsize: Figure size (width, height)
        cmap: Matplotlib colormap
    """
    history = ca.get_history()
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(history, cmap=cmap, interpolation='nearest', aspect='auto')
    ax.set_title(f'Rule {ca.rule}', fontsize=14)
    ax.set_xlabel('Cell Position')
    ax.set_ylabel('Generation')
    plt.tight_layout()
    plt.show()


def plot_rule_table(rule: int):
    """Visualize the rule lookup table."""
    fig, axes = plt.subplots(1, 8, figsize=(14, 2))
    fig.suptitle(f'Rule {rule} Transition Table', fontsize=12)
    
    for i, ax in enumerate(axes):
        # Neighborhoods are ordered 7,6,5,4,3,2,1,0 (111 to 000)
        idx = 7 - i
        output = (rule >> idx) & 1
        
        # Create a 2x3 grid showing input neighborhood and output
        grid = np.zeros((2, 3))
        grid[0, 0] = (idx >> 2) & 1  # left
        grid[0, 1] = (idx >> 1) & 1  # center
        grid[0, 2] = idx & 1         # right
        grid[1, 1] = output          # output (center of bottom row)
        
        ax.imshow(grid, cmap='binary', vmin=0, vmax=1)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'{idx:03b}→{output}', fontsize=10)
    
    plt.tight_layout()
    plt.show()

## Discrete Calculus Operations

Let's examine how the derivative, integral, and commutator work.

In [ ]:
# Demonstrate the calculus with Rule 30
ca = WolframCA(rule=30, width=21)
state = ca.initialize()

print("S  (state):     ", state)
print("D(S) (derivative):", ca.differentiate(state))
print("E(S) (evolved):   ", ca.evolve(state))
print()
print("Verify: E(S) = I(S, D(S)) = S XOR D(S)")
d = ca.differentiate(state)
print("I(S, D(S)):       ", ca.integrate(state, d))
print()
print("C(S) (commutator):", ca.commutator(state))
print(f"Nonlinearity count: {np.sum(ca.commutator(state))} cells")

In [ ]:
def plot_calculus_analysis(ca: WolframCA, generations: int = 50, figsize: tuple = (14, 10)):
    """
    Visualize state evolution alongside derivatives and commutators.
    
    Args:
        ca: WolframCA instance (should be initialized)
        generations: Number of generations to simulate
        figsize: Figure size
    """
    states = [ca.history[-1].copy()]
    derivatives = []
    commutators = []
    
    current = ca.history[-1].copy()
    for _ in range(generations):
        derivatives.append(ca.differentiate(current))
        commutators.append(ca.commutator(current))
        current = ca.evolve(current)
        states.append(current)
    
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    axes[0].imshow(states, cmap='binary', interpolation='nearest', aspect='auto')
    axes[0].set_title(f'State Evolution S', fontsize=12)
    axes[0].set_xlabel('Cell Position')
    axes[0].set_ylabel('Generation')
    
    axes[1].imshow(derivatives, cmap='binary', interpolation='nearest', aspect='auto')
    axes[1].set_title(f'Derivative D(S)', fontsize=12)
    axes[1].set_xlabel('Cell Position')
    axes[1].set_ylabel('Generation')
    
    axes[2].imshow(commutators, cmap='Reds', interpolation='nearest', aspect='auto')
    axes[2].set_title(f'Commutator C(S) - Nonlinearity', fontsize=12)
    axes[2].set_xlabel('Cell Position')
    axes[2].set_ylabel('Generation')
    
    plt.suptitle(f'Rule {ca.rule} Calculus Analysis', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    total_nonlinearity = sum(np.sum(c) for c in commutators)
    print(f"Total nonlinearity (sum of C): {total_nonlinearity}")

In [ ]:
# Visualize calculus for Rule 30 (chaotic)
ca30_calc = WolframCA(rule=30, width=101)
ca30_calc.initialize()
plot_calculus_analysis(ca30_calc, generations=50)

In [ ]:
# Compare with Rule 90 (linear XOR rule - Sierpinski triangle)
# Rule 90 is purely XOR-based, so D and E should commute!
ca90_calc = WolframCA(rule=90, width=101)
ca90_calc.initialize()
plot_calculus_analysis(ca90_calc, generations=50)

## Example: Rule 30

Rule 30 is famous for producing chaotic, aperiodic patterns from a simple initial condition. It has been used as a pseudorandom number generator.

In [ ]:
# Visualize Rule 30's transition table
plot_rule_table(30)

In [ ]:
# Run Rule 30
ca30 = WolframCA(rule=30, width=201)
ca30.initialize()
ca30.run(100)
plot_ca(ca30)

## Example: Rule 110

Rule 110 is proven to be Turing complete - it can simulate any computation given the right initial conditions.

In [ ]:
plot_rule_table(110)

In [ ]:
ca110 = WolframCA(rule=110, width=201)
ca110.initialize()
ca110.run(100)
plot_ca(ca110)

## Example: Rule 90

Rule 90 produces the Sierpiński triangle - a classic fractal pattern.

In [ ]:
plot_rule_table(90)

In [ ]:
ca90 = WolframCA(rule=90, width=201)
ca90.initialize()
ca90.run(100)
plot_ca(ca90)

## Interactive Exploration

Try different rules and initial conditions below.

In [ ]:
def explore_rule(rule: int, width: int = 201, generations: int = 100, 
                 random_init: bool = False, density: float = 0.5):
    """
    Explore a CA rule with customizable parameters.
    
    Args:
        rule: Rule number (0-255)
        width: Grid width
        generations: Number of generations to simulate
        random_init: If True, use random initial state
        density: Probability of each cell being 1 (if random_init=True)
    """
    ca = WolframCA(rule=rule, width=width)
    
    if random_init:
        initial = (np.random.random(width) < density).astype(np.uint8)
        ca.initialize(initial)
    else:
        ca.initialize()
    
    ca.run(generations)
    
    plot_rule_table(rule)
    plot_ca(ca)

In [ ]:
# Try Rule 184 (traffic flow model)
explore_rule(184, random_init=True, density=0.3)

In [ ]:
# Explore your own rule!
explore_rule(rule=73, width=201, generations=100)

## Rule Classification

Wolfram classified CA rules into four classes:

1. **Class I**: Evolution leads to homogeneous state (e.g., Rule 0, 32)
2. **Class II**: Evolution leads to periodic structures (e.g., Rule 4, 108)
3. **Class III**: Evolution leads to chaotic/aperiodic patterns (e.g., Rule 30, 45)
4. **Class IV**: Complex localized structures, long transients (e.g., Rule 110, 54)

In [ ]:
# Compare rules from different classes
class_examples = {
    'Class I (Homogeneous)': 32,
    'Class II (Periodic)': 4,
    'Class III (Chaotic)': 30,
    'Class IV (Complex)': 110
}

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, (class_name, rule) in zip(axes.flat, class_examples.items()):
    ca = WolframCA(rule=rule, width=151)
    ca.initialize()
    ca.run(75)
    
    ax.imshow(ca.get_history(), cmap='binary', interpolation='nearest', aspect='auto')
    ax.set_title(f'{class_name}: Rule {rule}', fontsize=11)
    ax.set_xlabel('Cell Position')
    ax.set_ylabel('Generation')

plt.tight_layout()
plt.show()

In [ ]:
def visualize_rule_dynamics(rule: int, width: int = 151, generations: int = 75, 
                             save_path: str = None, figsize: tuple = (12, 5)):
    """
    Visualize state evolution S and commutator C side by side for a given rule.
    
    Args:
        rule: Wolfram rule number (0-255)
        width: Grid width
        generations: Number of generations to simulate
        save_path: If provided, save the figure to this path
        figsize: Figure size (width, height)
    
    Returns:
        Tuple of (total_nonlinearity, fig) where total_nonlinearity is the sum of all commutator values
    """
    ca = WolframCA(rule=rule, width=width)
    ca.initialize()
    
    states = [ca.history[-1].copy()]
    commutators = []
    
    current = ca.history[-1].copy()
    for _ in range(generations):
        commutators.append(ca.commutator(current))
        current = ca.evolve(current)
        states.append(current)
    
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    
    # State evolution
    axes[0].imshow(states, cmap='binary', interpolation='nearest', aspect='auto')
    axes[0].set_title(f'Rule {rule}: State Evolution S', fontsize=12)
    axes[0].set_xlabel('Cell Position')
    axes[0].set_ylabel('Generation')
    
    # Commutator
    im = axes[1].imshow(commutators, cmap='Reds', interpolation='nearest', aspect='auto')
    axes[1].set_title(f'Rule {rule}: Commutator C (Nonlinearity)', fontsize=12)
    axes[1].set_xlabel('Cell Position')
    axes[1].set_ylabel('Generation')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"Saved to {save_path}")
    
    plt.show()
    
    total_nonlinearity = sum(np.sum(c) for c in commutators)
    return total_nonlinearity, fig


# Generate images for README
import os
os.makedirs('images', exist_ok=True)

# Select rules from different classes with interesting behavior
rules_to_visualize = [30, 90, 110, 184]

for rule in rules_to_visualize:
    nonlinearity, _ = visualize_rule_dynamics(rule, save_path=f'images/rule_{rule}.png')
    print(f"Rule {rule}: Total nonlinearity = {nonlinearity}\n")